## Categorise expenses

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import json
import os.path as path

### Files format and content

The following code was developed for Nationwide bank statements. It can be easily applied to other formats. Please make sure that the appropriate fields are changed. These parts are highlighted in the code.

**dictionary_training.json**: dictionary trained with expenses from previous year. {'Expense_name': [False, ..., True,..., False]}, where True corresponds to the category of the expense;

**expenses_train.csv**: contains the categorised expenses from the previous years. The first row is a comment, the second row is a list of the categories;

**account.csv**: this is the current year account to be categorised. The first four rows contain information that irrelevant for the taks. The columns are "Date,Transaction type,Description,Paid out,Paid in,Balance";

**sales.csv**: this files contains all the income entries for the year;

**review_expenses.csv**: the expenses that were not categorised in the previous years will end up in this file. Please take the time to categorise them by hand and re-run the code so that these will be assigned correctly for next year;

**total.csv**: this file contains both the sales and expenses combined in chronological order.

Before running the script please make sure that the files 'dictionary_training.json' (if available) and 'expenses_train.csv' is in the folder where this code is being run. If these two files are not present, the code will run, but all the expenses will end up in the "review_expenses.csv' file.

In [2]:
if path.exists("dictionary_training.json") :   
    with open('dictionary_training.json') as f: 
        data = f.read() 
    dic = json.loads(data) 
else:
    dic = {}
    
if path.exists('expenses_train.csv'):
    expenses = pd.read_csv('expenses_train.csv', skiprows=1, skip_blank_lines=True)
else:
    if len(dic) == 0:
        print('No previous dictionary and no data to train a new one are available. The program will continue with no categorisation')
    
    
for k,v in expenses.groupby('Description').groups.items():
    dic[k] = expenses.iloc[v.values[0]].notna().tolist()  

with open('dictionary_training.json', 'w') as d:
    json.dump(dic, d)    

In [3]:
#First income only
account = pd.read_csv('account.csv', skiprows=4, skip_blank_lines=True)
account = account.dropna(axis=0, how='all')
#Drop the balance (not needed for the categorisation)
account = account.drop('Balance', axis=1)
account = pd.concat([account,pd.DataFrame(columns=expenses.columns[5:])])

income = pd.DataFrame(columns=account.columns)
income['Date'] = pd.to_datetime(income['Date'])
for i in account.index.tolist():    
    if pd.notnull(account.loc[i]['Paid in']):
        income = income.append(account.loc[i])
        account = account.drop(labels=i)
#'Sales income' is one of the categories
income['Sales income'] = income['Paid in']

income[income.columns] = income[income.columns].replace({'£': ''}, regex=True)
income.to_csv('sales.csv', index=False)

In [4]:
#Categorise expenses
review_expenses = pd.DataFrame(columns=account.columns)
for i in range(account.shape[0]):    
    if account.iloc[i]['Description'] in dic:
        account.iloc[i,5:] = account.iloc[i,3]
        for j in range(len(account.columns)):
            account.iloc[i,j] = account.iloc[i,j] * dic[account.iloc[i]['Description']][j]
    else:
        review_expenses = review_expenses.append(account.iloc[i])
account.to_csv('categorised_expenses.csv')

In [6]:
#Combine expenses and sales
total = pd.concat([income,account])
total['Date'] = pd.to_datetime(total['Date'])
total = total.sort_values(by="Date")

total[total.columns] = total[total.columns].replace({'£': ''}, regex=True)
review_expenses[review_expenses.columns] = review_expenses[review_expenses.columns].replace({'£': ''}, regex=True)
total.to_csv('total.csv', index=False)
review_expenses.to_csv('review_expenses.csv', index=False)